Creator : Dhruuv Agarwal

Experiment/Hobby: Try to predict the outcome of NBA games before they take place. Aim is to learn and practice the complete workflow of a data science project while doing something interesting and in which domain knowledge is available.

The tasks involved:

1. Getting to know the data sources as there was no dataset known for this available.
2. Building the dataset by scraping.
3. Feature engineering from those scraped tables to have some valuable info.
4. Try different models to understand what model and features suit the problem.


    

In [464]:
import pandas as pd
from time import time
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from IPython.display import display
%matplotlib inline

Reading the 2016-17 best players obtained from scraping from basketball reference.com. These best players include players included in All star games and All Nba teams inclluding All Nba defensive team. This was considered to select the more valuable players by the league as the ones affecting results.

In [228]:
top17_df = pd.read_csv('2017_topplayers.csv',header=None)

In [229]:
# Examine the dataset with its initial formatting
top17_df.head()

,0,1,2
0,Anthony Davis,Patrick Beverley,Giannis Antetokounmpo
1,James Harden,Rudy Gobert,Carmelo Anthony
2,LeBron James,Draymond Green,Jimmy Butler
3,Kawhi Leonard,Kawhi Leonard,DeMar DeRozan
4,Russell Westbrook,Chris Paul,Paul George


In [230]:

top16_df = pd.read_csv('2016_topplayers.csv',header=None)
top15_df = pd.read_csv('2015_topplayers.csv',header=None)

Here, the list of best players are loaded in a similar fashion. Aim is to keep track of valuable players in the last 3 years, and with age less than 35.

In [231]:
tmp = [top17_df,top16_df,top15_df]
top=[]
[top.extend(x[y].tolist()) for x in tmp for y in [0,1,2]]

[None, None, None, None, None, None, None, None, None]

In [232]:
cleantop = [str(x).replace('\xa0','') for x in top]
cleantop

['Anthony Davis',
 'James Harden',
 'LeBron James',
 'Kawhi Leonard',
 'Russell Westbrook',
 'Giannis Antetokounmpo',
 'Stephen Curry',
 'Kevin Durant',
 'Rudy Gobert',
 'Isaiah Thomas',
 'Jimmy Butler',
 'DeMar DeRozan',
 'Draymond Green',
 'DeAndre Jordan',
 'John Wall',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'Patrick Beverley',
 'Rudy Gobert',
 'Draymond Green',
 'Kawhi Leonard',
 'Chris Paul',
 'Tony Allen',
 'Giannis Antetokounmpo',
 'Anthony Davis',
 'Danny Green',
 'Andre Roberson',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'Giannis Antetokounmpo',
 'Carmelo Anthony',
 'Jimmy Butler',
 'DeMar DeRozan',
 'Paul George',
 'Kyrie Irving',
 'LeBron James',
 'Kevin Love',
 'Kyle Lowry',
 'Paul Millsap',
 'Isaiah Thomas',
 'Kemba Walker',
 'John Wall',
 'DeMarcus Cousins',
 'Stephen Curry',
 'Anthony Davis',
 'Kevin Durant',
 'Marc Gasol',
 'Draymond Green',
 'James 

In [585]:
cleantop = list(set(cleantop))
if 'nan' in cleantop: cleantop.remove('nan')
cleantop[0:5]

['Tony Allen',
 'Andre Roberson',
 'Carmelo Anthony',
 'Kyrie Irving',
 'LeBron James']

In the below steps, The scraped team per game data from 2016-17 is used. The aim is to get the list of teams so that we can assign the players to them after that.

In [234]:
teams17_df = pd.read_csv('2017_team.csv')
teams17_df.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Golden State Warriors*,82,241.2,43.1,87.1,0.495,12.0,31.2,0.383,...,0.788,9.4,35.0,44.4,30.4,9.6,6.8,14.8,19.3,115.9
1,2.0,Houston Rockets*,82,241.2,40.3,87.2,0.462,14.4,40.3,0.357,...,0.766,10.9,33.5,44.4,25.2,8.2,4.3,15.1,19.9,115.3
2,3.0,Denver Nuggets,82,240.9,41.2,87.7,0.469,10.6,28.8,0.368,...,0.774,11.8,34.6,46.4,25.3,6.9,3.9,15.0,19.1,111.7
3,4.0,Cleveland Cavaliers*,82,242.4,39.9,84.9,0.470,13.0,33.9,0.384,...,0.748,9.3,34.4,43.7,22.7,6.6,4.0,13.7,18.1,110.3
4,5.0,Washington Wizards*,82,242.1,41.3,87.0,0.475,9.2,24.8,0.372,...,0.784,10.3,32.6,42.9,23.9,8.5,4.1,14.2,21.3,109.2


In [235]:
teams = teams17_df['Team'].tolist()
cleanteams =[x.replace('*','') for x in teams[:-1]]
cleanteams

['Golden State Warriors',
 'Houston Rockets',
 'Denver Nuggets',
 'Cleveland Cavaliers',
 'Washington Wizards',
 'Los Angeles Clippers',
 'Boston Celtics',
 'Portland Trail Blazers',
 'Phoenix Suns',
 'Toronto Raptors',
 'Oklahoma City Thunder',
 'Brooklyn Nets',
 'Minnesota Timberwolves',
 'San Antonio Spurs',
 'Indiana Pacers',
 'Charlotte Hornets',
 'Los Angeles Lakers',
 'New Orleans Pelicans',
 'New York Knicks',
 'Milwaukee Bucks',
 'Miami Heat',
 'Atlanta Hawks',
 'Chicago Bulls',
 'Sacramento Kings',
 'Philadelphia 76ers',
 'Detroit Pistons',
 'Orlando Magic',
 'Utah Jazz',
 'Memphis Grizzlies',
 'Dallas Mavericks']

In [236]:
abbr = ['GSW','HOU','DEN','CLE','WAS','LAC','BOS','POR','PHX','TOR','OKC','BRK','MIN','SAS','IND','CHO','LAL','NOP','NYK','MIL','MIA','ATL','CHI','SAC','PHI','DET','ORL','UTA','MEM','DAL']


Data frame for team and its abbreviations. Abbreviation was needed as in later steps for every match possible inactive or injured players would be scraped from basketall reference.

In [586]:
dfteam = pd.DataFrame({'Team':cleanteams,'abbr':abbr})

In [238]:
dfteam

,Team,abbr
0,Golden State Warriors,GSW
1,Houston Rockets,HOU
2,Denver Nuggets,DEN
3,Cleveland Cavaliers,CLE
4,Washington Wizards,WAS
5,Los Angeles Clippers,LAC
6,Boston Celtics,BOS
7,Portland Trail Blazers,POR
8,Phoenix Suns,PHX
9,Toronto Raptors,TOR


Below is the code for scrpaing for a mathc and its inactive players. The address contained a portion unique for date +0+ teamabbr; this was needed to be generated every time to get the required data.

In [587]:
from bs4 import BeautifulSoup,Comment
import requests, re

def get_url(team, year):
    link = "https://www.basketball-reference.com/teams/"+str(team)+"/" + str(year) +".html"
    return link
 
 
    
def main_scraping(adr):
     
    response = requests.get("https://www.basketball-reference.com/boxscores/"+adr+'.html')
    
    content = response.content.decode(encoding='UTF-8')
    soup = BeautifulSoup(content, "html.parser")
    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    [comment.extract() for comment in comments]
    containers2 = soup.find_all("div")
    lst=[]
    for container in containers2[:]:
        if "Inactive:" in container.text : 
            lst = container.text.partition("Inactive:")[2].partition("Officials")[0]
            break
    return(lst)
main_scraping('201610250CLE')        


'\xa0\nNYK\xa0Maurice Ndour,\xa0Marshall Plumlee\xa0\xa0\xa0CLE\xa0Kay Felder,\xa0Mo Williams\xa0\xa0\xa0\n'

Gettiing the 2016-17 and 2017-18 results data from the scraped files to load as dataframes.

In [349]:
results17 = pd.read_csv('2017_results.csv')
results17.head()
results17.rename(columns = {'Home/Neutral':'Home','Visitor/Neutral':'Away'}, inplace = True)
results17["Date"][0]

'Tue Oct 25 2016'

In [262]:
results18 = pd.read_csv('2018_results.csv')
results18.head()
results18.rename(columns = {'Home/Neutral':'Home','Visitor/Neutral':'Away'}, inplace = True)
results18.head()

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,Tue Oct 17 2017,8:01p,Boston Celtics,99,Cleveland Cavaliers,102,Box Score,NaN,20562,NaN
1,Tue Oct 17 2017,10:30p,Houston Rockets,122,Golden State Warriors,121,Box Score,NaN,19596,NaN
2,Wed Oct 18 2017,7:30p,Milwaukee Bucks,108,Boston Celtics,100,Box Score,NaN,18624,NaN
3,Wed Oct 18 2017,8:30p,Atlanta Hawks,117,Dallas Mavericks,111,Box Score,NaN,19709,NaN
4,Wed Oct 18 2017,7:00p,Charlotte Hornets,90,Detroit Pistons,102,Box Score,NaN,20491,NaN


Below an element from Date column is taken to get the required info , to check if scraping address info is possible to get from here.

In [241]:
from datetime import datetime
dateobject = datetime.strptime(results17["Date"][0], '%a %b %d %Y')
dfteam.Team[dfteam.Team == results17["Home"][0]].index.tolist()
sel_abbr = dfteam.Team[dfteam.Team == results17["Home"][0]].index.tolist()
dateobject.strftime('%Y%m%d') +'0'+ str(dfteam.abbr[sel_abbr[0]])

2016-10-25 00:00:00


'201610250CLE'

Here, we get the list of players we sorted out before assigned to respective teams in 2016-17 season, to know which stars are in which teams.

In [303]:
stars = ['Klay Thompson,Kevin Durant,Stephen Curry,Draymond Green','James Harden,Patrick Beverley','','LeBron James,Kyrie Irving,Kevin Love','John Wall,Bradley Beal','DeAndre Jordan,Chris Paul,Blake Griffin','Avery Bradley,Isaiah Thomas,Al Horford','Damian Lillard','','Kyle Lowry,DeMar DeRozan','Russell Westbrook,Andre Roberson','','','Kawhi Leonard,LaMarcus Aldridge,Danny Green','Paul George,Jeff Teague','Kemba Walker','','DeMarcus Cousins,Anthony Davis','Carmelo Anthony','Giannis Antetokounmpo','Hassan Whiteside','Paul Millsap','Jimmy Butler','DeMarcus Cousins','','Andre Drummond','','Gordon Hayward,Rudy Gobert','Marc Gasol','']
s = pd.Series(stars, index=dfteam.index)
stars18 = ['Klay Thompson,Kevin Durant,Stephen Curry,Draymond Green','James Harden,Chris Paul','Paul Millsap','LeBron James,Isaiah Thomas,Kevin Love','John Wall,Bradley Beal','DeAndre Jordan,Patrick Beverley,Blake Griffin','Al Horford,Kyrie Irving','Damian Lillard','','Kyle Lowry,DeMar DeRozan','Russell Westbrook,Andre Roberson,Paul George,Carmelo Anthony','','Jeff Teague,Jimmy Butler','Kawhi Leonard,LaMarcus Aldridge,Danny Green','','Kemba Walker','Isaiah Thomas','DeMarcus Cousins,Anthony Davis','','Giannis Antetokounmpo','Hassan Whiteside','','','','','Blake Griffin,Andre Drummond,Avery Bradley','','Rudy Gobert','Marc Gasol','']
dfteam17 = dfteam.assign(starplayers=s.values)
dfteam17
s2 = pd.Series(stars18, index=dfteam.index)
dfteam18 = dfteam.assign(starplayers=s2.values)
dfteam18

,Team,abbr,starplayers
0,Golden State Warriors,GSW,"Klay Thompson,Kevin Durant,Stephen Curry,Draym..."
1,Houston Rockets,HOU,"James Harden,Chris Paul"
2,Denver Nuggets,DEN,Paul Millsap
3,Cleveland Cavaliers,CLE,"LeBron James,Isaiah Thomas,Kevin Love"
4,Washington Wizards,WAS,"John Wall,Bradley Beal"
5,Los Angeles Clippers,LAC,"DeAndre Jordan,Patrick Beverley,Blake Griffin"
6,Boston Celtics,BOS,"Al Horford,Kyrie Irving"
7,Portland Trail Blazers,POR,Damian Lillard
8,Phoenix Suns,PHX,
9,Toronto Raptors,TOR,"Kyle Lowry,DeMar DeRozan"


To have a simple representation for training the model, we have count of stars as another column to the dataframe. We do this for both 2017 and 2018, train and test datasets.

In [304]:
def countpl(x):
    count = 0
    if x is not '':
        count =1 +x.count(',')
    return count    
        
dfteam17['star'] = dfteam17['starplayers'].apply(countpl)
dfteam17
dfteam18['star'] = dfteam18['starplayers'].apply(countpl)
dfteam18

,Team,abbr,starplayers,star
0,Golden State Warriors,GSW,"Klay Thompson,Kevin Durant,Stephen Curry,Draym...",4
1,Houston Rockets,HOU,"James Harden,Chris Paul",2
2,Denver Nuggets,DEN,Paul Millsap,1
3,Cleveland Cavaliers,CLE,"LeBron James,Isaiah Thomas,Kevin Love",3
4,Washington Wizards,WAS,"John Wall,Bradley Beal",2
5,Los Angeles Clippers,LAC,"DeAndre Jordan,Patrick Beverley,Blake Griffin",3
6,Boston Celtics,BOS,"Al Horford,Kyrie Irving",2
7,Portland Trail Blazers,POR,Damian Lillard,1
8,Phoenix Suns,PHX,,0
9,Toronto Raptors,TOR,"Kyle Lowry,DeMar DeRozan",2


After getting the player-team-star info for the 2017 and 2018 seasons, now we need to incorporate this info as a column/feature in the results dataset. This is done, by using apply function on the dataframe along the row axis. What this does is use the Dat entry in each row, modeify it to get the link address string required for scraping. This checks for inactive players online by scraping. After getting a list of inactive players, we compare it with the stars of the team, to know finally how many stars are available to the team. 

In [335]:
def countpl(x):
    count = 0
    if x.rstrip() is not '':
        count =1 +x.count(',')
    return count
teamdfs = [dfteam17,dfteam18]
first = results18.Date[0]
firstdate = datetime.strptime(first, '%a %b %d %Y')
print(firstdate)
def starscount(df):
    d = df['Date']
    x = df['Home']
    y = df['Away']
    dateobject = datetime.strptime(d, '%a %b %d %Y')
    team = dfteam17 if (dateobject <firstdate) ==True else dfteam18
    sel_team = team.Team[team['Team']==x].index.tolist()
    y_team = team.Team[team['Team']==y].index.tolist()
    dateobject = datetime.strptime(d, '%a %b %d %Y')
    adr = dateobject.strftime('%Y%m%d') +'0'+ team.abbr[sel_team].values[0]
    lst = main_scraping(adr)
    home,homestar,away,awaystar = team.Team[sel_team].values[0],str(team.starplayers[sel_team].values[0]),team.Team[y_team].values[0],dfteam17.starplayers[y_team].values[0]
    hstar = homestar.split(',')
    astar = awaystar.split(',')
    df['Home stars'] = team.star[team.Team==home].values[0] - sum([lst.count(x) for x in hstar]) if countpl(homestar)!=0 else 0
    df['Away stars'] = team.star[team.Team==away].values[0] - sum([lst.count(x) for x in astar]) if countpl(awaystar)!=0 else 0
    return df#(neth,neta)




2017-10-17 00:00:00


In [352]:
results17 =  results17.apply(starscount,axis=1)
results17.head()

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars
0,Tue Oct 25 2016,7:30p,New York Knicks,88,Cleveland Cavaliers,117,Box Score,NaN,20562,NaN,3,1
1,Tue Oct 25 2016,10:30p,San Antonio Spurs,129,Golden State Warriors,100,Box Score,NaN,19596,NaN,4,2
2,Tue Oct 25 2016,10:00p,Utah Jazz,104,Portland Trail Blazers,113,Box Score,NaN,19446,NaN,1,1
3,Wed Oct 26 2016,7:30p,Brooklyn Nets,117,Boston Celtics,122,Box Score,NaN,18624,NaN,3,0
4,Wed Oct 26 2016,7:00p,Dallas Mavericks,121,Indiana Pacers,130,Box Score,OT,17923,NaN,2,0


In [337]:
results18 =  results18.apply(starscount,axis=1)
results18

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars
0,Tue Oct 17 2017,8:01p,Boston Celtics,99,Cleveland Cavaliers,102,Box Score,NaN,20562,NaN,2,1
1,Tue Oct 17 2017,10:30p,Houston Rockets,122,Golden State Warriors,121,Box Score,NaN,19596,NaN,4,2
2,Wed Oct 18 2017,7:30p,Milwaukee Bucks,108,Boston Celtics,100,Box Score,NaN,18624,NaN,2,1
3,Wed Oct 18 2017,8:30p,Atlanta Hawks,117,Dallas Mavericks,111,Box Score,NaN,19709,NaN,0,0
4,Wed Oct 18 2017,7:00p,Charlotte Hornets,90,Detroit Pistons,102,Box Score,NaN,20491,NaN,3,1
5,Wed Oct 18 2017,7:00p,Brooklyn Nets,131,Indiana Pacers,140,Box Score,NaN,15008,NaN,0,0
6,Wed Oct 18 2017,8:00p,New Orleans Pelicans,91,Memphis Grizzlies,103,Box Score,NaN,17794,NaN,1,2
7,Wed Oct 18 2017,7:00p,Miami Heat,109,Orlando Magic,116,Box Score,NaN,18846,NaN,0,1
8,Wed Oct 18 2017,10:00p,Portland Trail Blazers,124,Phoenix Suns,76,Box Score,NaN,18055,NaN,0,1
9,Wed Oct 18 2017,10:00p,Houston Rockets,105,Sacramento Kings,100,Box Score,NaN,17583,NaN,0,2


In [429]:
#stars = ['Klay Thompson,Kevin Durant,Stephen Curry,Draymond Green','James Harden,Patrick Beverley','','LeBron James,Kyrie Irving,Kevin Love','John Wall,Bradley Beal','DeAndre Jordan,Chris Paul,Blake Griffin','Avery Bradley,Isaiah Thomas,Al Horford','Damian Lillard','','Kyle Lowry,DeMar DeRozan','Russell Westbrook,Andre Roberson','','','Kawhi Leonard,LaMarcus Aldridge,Danny Green','Paul George,Jeff Teague','Kemba Walker','','DeMarcus Cousins,Anthony Davis','Carmelo Anthony','Giannis Antetokounmpo','Hassan Whiteside','Paul Millsap','Jimmy Butler','DeMarcus Cousins','','Andre Drummond','','Gordon Hayward,Rudy Gobert','Marc Gasol','']
#dfteam
#results17=results17backup
#results17backup = results17
results17

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars,Home Win
0,Tue Oct 25 2016,7:30p,New York Knicks,88,Cleveland Cavaliers,117,Box Score,NaN,20562,NaN,3,1,True
1,Tue Oct 25 2016,10:30p,San Antonio Spurs,129,Golden State Warriors,100,Box Score,NaN,19596,NaN,4,2,False
2,Tue Oct 25 2016,10:00p,Utah Jazz,104,Portland Trail Blazers,113,Box Score,NaN,19446,NaN,1,1,True
3,Wed Oct 26 2016,7:30p,Brooklyn Nets,117,Boston Celtics,122,Box Score,NaN,18624,NaN,3,0,True
4,Wed Oct 26 2016,7:00p,Dallas Mavericks,121,Indiana Pacers,130,Box Score,OT,17923,NaN,2,0,True
5,Wed Oct 26 2016,10:30p,Houston Rockets,114,Los Angeles Lakers,120,Box Score,NaN,18997,NaN,0,1,True
6,Wed Oct 26 2016,8:00p,Minnesota Timberwolves,98,Memphis Grizzlies,102,Box Score,NaN,18119,NaN,1,0,True
7,Wed Oct 26 2016,8:00p,Charlotte Hornets,107,Milwaukee Bucks,96,Box Score,NaN,18717,NaN,1,1,False
8,Wed Oct 26 2016,8:00p,Denver Nuggets,107,New Orleans Pelicans,102,Box Score,NaN,15869,NaN,2,0,False
9,Wed Oct 26 2016,7:00p,Miami Heat,108,Orlando Magic,96,Box Score,NaN,19298,NaN,0,1,False


In [402]:
from sklearn.metrics import f1_score, make_scorer, classification_report

scorer = make_scorer(f1_score, pos_label = None, average = 'weighted')

Making the y_truth column in both the test and train datasets

In [473]:
results17['Home Win'] = results17['PTS'] < results17['PTS.1']
results18['Home Win'] = results18['PTS'] < results18['PTS.1']
### This will also be the value that we want to predict ###
y_true = results17['Home Win'].values
y_test = results18['Home Win'].values

In [405]:
results17.head()

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars,Home Win
0,Tue Oct 25 2016,7:30p,New York Knicks,88,Cleveland Cavaliers,117,Box Score,NaN,20562,NaN,3,1,True
1,Tue Oct 25 2016,10:30p,San Antonio Spurs,129,Golden State Warriors,100,Box Score,NaN,19596,NaN,4,2,False
2,Tue Oct 25 2016,10:00p,Utah Jazz,104,Portland Trail Blazers,113,Box Score,NaN,19446,NaN,1,1,True
3,Wed Oct 26 2016,7:30p,Brooklyn Nets,117,Boston Celtics,122,Box Score,NaN,18624,NaN,3,0,True
4,Wed Oct 26 2016,7:00p,Dallas Mavericks,121,Indiana Pacers,130,Box Score,OT,17923,NaN,2,0,True


The baseline is the win percentage for a normal Home team. Unlike general cases, where we have 50% chance of both cases, sports have concept of Home-Away games which traditionally have different outcomes.

In [387]:
# Establishing the baseline win percentage for the Home Teams overall
games = results17['Home Win'].count()
homewins = results17['Home Win'].sum()
win_percentage = homewins / games

print('Home Win percentage: {0:.2f}%'.format(100 * win_percentage))

Home Win percentage: 58.29%


In [388]:
# Predicting the baseline for Home Teams wins with simple classifier

from sklearn.metrics import f1_score

y_pred = [1] * len(y_true)
print('F1: {0:.4f}%'.format(f1_score(y_true, y_pred, pos_label = None, average = 'weighted') * 100))

F1: 42.9289%


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The scores come out to be decent , 58 percent accuracy and 42 as f1 score.

Below we explore another possible feature to incorporate the last results for both away and home teams. Home last and Away last show if the team won its last match

In [475]:
teamlast = {x:False for x in dfteam.Team.values}
def checklastwin(df):
    x = df['Home']
    y = df['Away']
    #print(teamlast,x)
    
    df['Home Last'] = teamlast[x]
    df['Away Last'] = teamlast[y]
    if df['Home Win'] : 
        teamlast[x] = True
        teamlast[y] = False
    else :  
        teamlast[y]=True
        teamlast[x]=False
    #print(df['Home Last'])
    return df   
results17 = results17.apply(checklastwin,axis=1)
results18 = results18.apply(checklastwin,axis=1)
results17.head()

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars,Home Win,Home Last,Away Last
0,Tue Oct 25 2016,7:30p,New York Knicks,88,Cleveland Cavaliers,117,Box Score,NaN,20562,NaN,3,1,True,True,False
1,Tue Oct 25 2016,10:30p,San Antonio Spurs,129,Golden State Warriors,100,Box Score,NaN,19596,NaN,4,2,False,False,False
2,Tue Oct 25 2016,10:00p,Utah Jazz,104,Portland Trail Blazers,113,Box Score,NaN,19446,NaN,1,1,True,False,False
3,Wed Oct 26 2016,7:30p,Brooklyn Nets,117,Boston Celtics,122,Box Score,NaN,18624,NaN,3,0,True,False,False
4,Wed Oct 26 2016,7:00p,Dallas Mavericks,121,Indiana Pacers,130,Box Score,OT,17923,NaN,2,0,True,False,False


Below, we add another possible feature/column to the dataframe. This incorporates the past match winning streak for both teams.

In [588]:
win = {x:0 for x in dfteam.Team.values}
runn2=0
def checkstreak(df):
    global runn2
    x = df['Home']
    y = df['Away']
    #print(teamlast,x)
    if runn2==1: 
        if df['Home Win'] : 
            win[x] -= 1
            win[y] = 0
        else :  
            win[y]-=1
            win[x]=0

    #print(runn2)
    df['Home winstreak'] = win[x]
    df['Away winstreak'] = win[y]
    
    if df['Home Win'] : 
        win[x] += 1
        win[y] = 0
    else :  
        win[y]+=1
        win[x]=0
    #print(df['Home Last'])
    runn2+=1
    return df   
results17 = results17.apply(checkstreak,axis=1)
runn2=0
win = {x:0 for x in dfteam.Team.values}
results18 = results18.apply(checkstreak,axis=1)
results17

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars,Home Win,Home Last,Away Last,Home Avg,Away Avg,Home winstreak,Away winstreak
0,Tue Oct 25 2016,7:30p,New York Knicks,88,Cleveland Cavaliers,117,Box Score,NaN,20562,NaN,3,1,True,False,False,0.000000,0.000000,0,0
1,Tue Oct 25 2016,10:30p,San Antonio Spurs,129,Golden State Warriors,100,Box Score,NaN,19596,NaN,4,2,False,True,False,0.000000,0.000000,0,0
2,Tue Oct 25 2016,10:00p,Utah Jazz,104,Portland Trail Blazers,113,Box Score,NaN,19446,NaN,1,1,True,False,False,0.000000,0.000000,0,0
3,Wed Oct 26 2016,7:30p,Brooklyn Nets,117,Boston Celtics,122,Box Score,NaN,18624,NaN,3,0,True,False,False,0.000000,0.000000,0,0
4,Wed Oct 26 2016,7:00p,Dallas Mavericks,121,Indiana Pacers,130,Box Score,OT,17923,NaN,2,0,True,False,False,0.000000,0.000000,0,0
5,Wed Oct 26 2016,10:30p,Houston Rockets,114,Los Angeles Lakers,120,Box Score,NaN,18997,NaN,0,1,True,True,False,0.000000,0.000000,0,0
6,Wed Oct 26 2016,8:00p,Minnesota Timberwolves,98,Memphis Grizzlies,102,Box Score,NaN,18119,NaN,1,0,True,False,False,0.000000,0.000000,0,0
7,Wed Oct 26 2016,8:00p,Charlotte Hornets,107,Milwaukee Bucks,96,Box Score,NaN,18717,NaN,1,1,False,False,True,0.000000,0.000000,0,0
8,Wed Oct 26 2016,8:00p,Denver Nuggets,107,New Orleans Pelicans,102,Box Score,NaN,15869,NaN,2,0,False,False,False,0.000000,0.000000,0,0
9,Wed Oct 26 2016,7:00p,Miami Heat,108,Orlando Magic,96,Box Score,NaN,19298,NaN,0,1,False,True,False,0.000000,0.000000,0,0


Below, we add another possible feature/column to the dataframe. This incorporates the past 5 match scoring averages for each team. The value represnt avg(pts scored-pts given) over 5 matches. 

In [548]:
teamscore = {x:[100] for x in dfteam.Team.values}
oppscore = {x:[100] for x in dfteam.Team.values}
print(teamscore["Utah Jazz"])
runn=0
def avgscore(df):
    global runn

    #print(runn)
    x = df['Home']
    y = df['Away']
    #print(teamlast,x)
    if(runn==1):
        teamscore[x].pop()
        oppscore[x].pop()
        teamscore[y].pop()
        oppscore[y].pop()

    df['Home Avg'] = np.mean(teamscore[x])-np.mean(oppscore[x])
    df['Away Avg'] = np.mean(teamscore[y]) - np.mean(oppscore[y])
    
    #print(df["PTS.1"])
    teamscore[x].append(df["PTS.1"])
    oppscore[x].append(df['PTS'])
    teamscore[y].append(df['PTS'])
    oppscore[y].append(df["PTS.1"])
    if len(teamscore[x])==6: 
        teamscore[x].pop(0)
        oppscore[x].pop(0)
    if len(teamscore[y])==6: 
        teamscore[y].pop(0)
        oppscore[y].pop(0)    
    #print(df['Home Last'])
    runn+=1
    return df

results17 = results17.apply(avgscore,axis=1)
runn=0
teamscore = {x:[100] for x in dfteam.Team.values}
oppscore = {x:[100] for x in dfteam.Team.values}
#print(runn,results17.shape)
results18 = results18.apply(avgscore,axis=1)
results17.head()

[100]


,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars,Home Win,Home Last,Away Last,Home Avg,Away Avg,Home winstreak,Away winstreak
0,Tue Oct 25 2016,7:30p,New York Knicks,88,Cleveland Cavaliers,117,Box Score,NaN,20562,NaN,3,1,True,False,False,0.0,0.0,0,0
1,Tue Oct 25 2016,10:30p,San Antonio Spurs,129,Golden State Warriors,100,Box Score,NaN,19596,NaN,4,2,False,True,False,0.0,0.0,0,0
2,Tue Oct 25 2016,10:00p,Utah Jazz,104,Portland Trail Blazers,113,Box Score,NaN,19446,NaN,1,1,True,False,False,0.0,0.0,0,0
3,Wed Oct 26 2016,7:30p,Brooklyn Nets,117,Boston Celtics,122,Box Score,NaN,18624,NaN,3,0,True,False,False,0.0,0.0,0,0
4,Wed Oct 26 2016,7:00p,Dallas Mavericks,121,Indiana Pacers,130,Box Score,OT,17923,NaN,2,0,True,False,False,0.0,0.0,0,0


In [549]:
results17.at[0,'Home Last']=False
results18.at[0,'Home Last']=False
results18

,Date,Start (ET),Away,PTS,Home,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Home stars,Away stars,Home Win,Home Last,Away Last,Home Avg,Away Avg,Home winstreak,Away winstreak
0,Tue Oct 17 2017,8:01p,Boston Celtics,99,Cleveland Cavaliers,102,Box Score,NaN,20562,NaN,2,1,True,False,False,0.000000,0.000000,0,0
1,Tue Oct 17 2017,10:30p,Houston Rockets,122,Golden State Warriors,121,Box Score,NaN,19596,NaN,4,2,False,True,False,0.000000,0.000000,0,0
2,Wed Oct 18 2017,7:30p,Milwaukee Bucks,108,Boston Celtics,100,Box Score,NaN,18624,NaN,2,1,False,False,False,-1.500000,0.000000,0,0
3,Wed Oct 18 2017,8:30p,Atlanta Hawks,117,Dallas Mavericks,111,Box Score,NaN,19709,NaN,0,0,False,True,False,0.000000,0.000000,0,0
4,Wed Oct 18 2017,7:00p,Charlotte Hornets,90,Detroit Pistons,102,Box Score,NaN,20491,NaN,3,1,True,False,False,0.000000,0.000000,0,0
5,Wed Oct 18 2017,7:00p,Brooklyn Nets,131,Indiana Pacers,140,Box Score,NaN,15008,NaN,0,0,True,False,False,0.000000,0.000000,0,0
6,Wed Oct 18 2017,8:00p,New Orleans Pelicans,91,Memphis Grizzlies,103,Box Score,NaN,17794,NaN,1,2,True,False,True,0.000000,0.000000,0,0
7,Wed Oct 18 2017,7:00p,Miami Heat,109,Orlando Magic,116,Box Score,NaN,18846,NaN,0,1,True,True,True,0.000000,0.000000,0,0
8,Wed Oct 18 2017,10:00p,Portland Trail Blazers,124,Phoenix Suns,76,Box Score,NaN,18055,NaN,0,1,False,False,False,0.000000,0.000000,0,0
9,Wed Oct 18 2017,10:00p,Houston Rockets,105,Sacramento Kings,100,Box Score,NaN,17583,NaN,0,2,False,False,True,0.000000,0.500000,0,1


In [467]:

def train_classifier(clf, X_train, y_train):
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred, pos_label=True), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [434]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 14) 

In [440]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Use selected features as input for the classifier (target)
X_previouswins = results17[['Home Last', 'Away Last']].values

# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_previouswins, y_true, scoring = scorer)

# Print results
print("Using just the last result from the home and visitor teams")
print('F1: {0:.4f}%'.format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
F1: 42.9288%


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in lab

We see that just having previous game win/loss info about the teams doesnt give much improvement alone.

In [561]:
X_previouswins = results17[['Home Last', 'Away Last','Home stars','Away stars']].values
X_previoustest = results18[['Home Last', 'Away Last','Home stars','Away stars']].values
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_previouswins, y_true, scoring = scorer)
dtc.fit(X_previouswins,y_true)
# Print results
print("Using just the last result from the home and visitor teams")
print('F1: {0:.4f}%'.format(np.mean(scores) * 100))
predict_labels(dtc,X_previoustest,y_test)

Using just the last result from the home and visitor teams
F1: 66.0706%
Made predictions in 0.0000 seconds.


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


(0.6914826498422713, 0.6272865853658537)

We see just the home and away stars info gives a drastic improvement in the f1 score.

In [598]:

X_previouswins = results17[['Home winstreak','Away winstreak']].values
X_previoustest = results18[['Home winstreak','Away winstreak']].values

from sklearn.preprocessing import scale


#X_previouswins[:,2:] = scale(X_previouswins[:,2:])
#X_previoustest[:,2:] = scale(X_previoustest[:,2:])
    
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_previouswins, y_true, scoring = scorer)
dtc.fit(X_previouswins,y_true)
# Print results
print("Using just the last result from the home and visitor teams")
print('F1: {0:.4f}%'.format(np.mean(scores) * 100))
predict_labels(dtc,X_previoustest,y_test)


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Using just the last result from the home and visitor teams
F1: 52.4319%
Made predictions in 0.0000 seconds.


(0.7070496083550915, 0.5724085365853658)

We also see that the Win streaks matter a lot to have a good prediction.

In [602]:
X_previouswins = results17[['Home Last', 'Away Last','Home stars','Away stars']].values
X_previoustest = results18[['Home Last', 'Away Last','Home stars','Away stars']].values

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameter_space = {
                   "max_features": ['sqrt'],
                   "n_estimators": [50, 100, 200,300],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [1, 2, 4, 6],
                   }

# Random Forest Classifier
rfc = RandomForestClassifier(random_state = 14)
grid = GridSearchCV(rfc, parameter_space, scoring = scorer)
grid.fit(X_previouswins, y_true)
# Print results
scores = cross_val_score(rfc, X_previouswins, y_true, scoring = scorer)
print("Using full team labels is ranked higher")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))
ypred = grid.predict(X_previoustest)
ytest = results18['Home Win']
print(classification_report(ypred,ytest))
predict_labels(grid,X_previoustest,y_test)

C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Using full team labels is ranked higher
F1: 64.6879%
              precision    recall  f1-score   support

       False       0.52      0.55      0.54       511
        True       0.70      0.68      0.69       801

   micro avg       0.63      0.63      0.63      1312
   macro avg       0.61      0.61      0.61      1312
weighted avg       0.63      0.63      0.63      1312

Made predictions in 0.0478 seconds.


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 10

(0.6887332908975174, 0.6272865853658537)

In [484]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

encoding = LabelEncoder()
encoding.fit(results17["Home"].values)
home_teams = encoding.transform(results17["Home"].values)
visitor_teams = encoding.transform(results17["Away"].values)

# Stacks arrays in a sequence vertically (row-wise) & transposes the data
X_teams = np.vstack([home_teams, visitor_teams]).T
onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()
X_teams.shape
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_teams, y_true, scoring = scorer)

# Print results
print("F1: {0:.4f}%".format(np.mean(scores) * 100))


Thome_teams = encoding.transform(results18["Home"].values)
Tvisitor_teams = encoding.transform(results18["Away"].values)

# Stacks arrays in a sequence vertically (row-wise) & transposes the data
X_teamstest = np.vstack([Thome_teams, Tvisitor_teams]).T
onehot = OneHotEncoder()
X_teamstest = onehot.fit_transform(X_teamstest).todense()


F1: 58.6740%


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\_e

In this experimentation with team encoding we see that other features like star info dominate the info gained by encoding.

In [485]:
parameter_space = {
                   "max_features": [2, 10, 50, 'auto'],
                   "n_estimators": [50, 100, 200],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [1, 2, 4, 6],
                   }

# Random Forest Classifier
rfc = RandomForestClassifier(random_state = 14)
grid = GridSearchCV(rfc, parameter_space, scoring = scorer)
grid.fit(X_teams, y_true)

# Print results
print("F1: {0:.4f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)
predict_labels(grid.best_estimator_,X_teamstest,y_test)

C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


F1: 62.5951%
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False)
Made predictions in 0.0312 seconds.


(0.6991473812423873, 0.6234756097560976)

In [450]:
X_all = np.hstack([X_previouswins, X_teams])
print(X_all.shape)

(1309, 64)


In [452]:
rfc = RandomForestClassifier(random_state = 14)
grid = GridSearchCV(rfc, parameter_space, scoring = scorer)
grid.fit(X_all, y_true)

# Print results
print("F1: {0:.4f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


F1: 63.7492%
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=50, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False)


In [583]:
X_previouswins = results17[['Home Last', 'Home stars','Away stars','Away Last','Home winstreak','Away winstreak']].values
X_previoustest = results18[['Home Last', 'Home stars','Away stars','Away Last','Home winstreak','Away winstreak']].values

clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 912, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)


#predict_labels(grid.best_estimator_,X_previoustest,y_test)

train_predict(clf_A,X_previouswins,y_true,X_previoustest,y_test)
train_predict(clf_B,X_previouswins,y_true,X_previoustest,y_test)
train_predict(clf_C,X_previouswins,y_true,X_previoustest,y_test)


Training a LogisticRegression using a training set size of 1309. . .
Trained model in 0.0060 seconds
Made predictions in 0.0010 seconds.
0.7169811320754716 0.6562261268143621
F1 score and accuracy score for training set: 0.7170 , 0.6562.
Made predictions in 0.0010 seconds.
F1 score and accuracy score for test set: 0.6991 , 0.6280.
Training a SVC using a training set size of 1309. . .
Trained model in 0.0399 seconds
Made predictions in 0.0210 seconds.
0.7465712581991651 0.6753246753246753
F1 score and accuracy score for training set: 0.7466 , 0.6753.
Made predictions in 0.0209 seconds.
F1 score and accuracy score for test set: 0.7278 , 0.6448.
Training a XGBClassifier using a training set size of 1309. . .
Trained model in 0.0588 seconds
Made predictions in 0.0021 seconds.
0.7458665033680343 0.6829640947288006
F1 score and accuracy score for training set: 0.7459 , 0.6830.
Made predictions in 0.0020 seconds.
F1 score and accuracy score for test set: 0.7058 , 0.6258.


C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\dhruu\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [481]:
parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label=True)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_previouswins,y_true)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_previouswins, y_true)
print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
f1, acc = predict_labels(clf, X_previoustest, y_test)
print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.0000 seconds.
F1 score and accuracy score for training set: 0.7267 , 0.6662.
Made predictions in 0.0000 seconds.
F1 score and accuracy score for test set: 0.7016 , 0.6311.
